# Intro

For generating dataset of shape (n_samples, max_seq_len, feats_per_step) from the original dataframe 

This version is the default

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

import pickle
from datetime import date

In [4]:
RAW_CSV_DIR = "raw_czech"

for fname in os.listdir(RAW_CSV_DIR):
    if ".csv" in fname and "new_" in fname:
        print(fname)
        csv_name = fname.split(".")[0]
        cmd = f"{csv_name} = pd.read_csv('{RAW_CSV_DIR}/{fname}')"
        print("Running:", cmd)
        exec(cmd)


new_account.csv
Running: new_account = pd.read_csv('raw_czech/new_account.csv')
new_disposition.csv
Running: new_disposition = pd.read_csv('raw_czech/new_disposition.csv')
new_transaction.csv
Running: new_transaction = pd.read_csv('raw_czech/new_transaction.csv')
new_card.csv
Running: new_card = pd.read_csv('raw_czech/new_card.csv')
new_client.csv
Running: new_client = pd.read_csv('raw_czech/new_client.csv')


### Setup

### Sort by acct 

In [5]:
tr_by_acct = new_transaction.sort_values(by = ["account_id", "date"])

In [6]:
diffs = np.diff(tr_by_acct["account_id"]) > 0   # detect acct change
np.where(diffs)

(array([    238,     716,     833, ..., 1055344, 1055722, 1056066]),)

In [7]:
tr_by_acct

,Unnamed: 0,account_id,date,type,operation,amount,balance,k_symbol
149431,149432,1,950324,CREDIT,CREDIT IN CASH,1000.0,1000.0,NaN
157403,157404,1,950413,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,4679.0,NaN
158831,158832,1,950423,CREDIT,CREDIT IN CASH,12600.0,17279.0,NaN
162680,162681,1,950430,CREDIT,NaN,19.2,17298.2,INTEREST CREDITED
167082,167083,1,950513,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,20977.2,NaN
...,...,...,...,...,...,...,...,...
1033140,1033141,11382,981202,DEBIT,CASH WITHDRAWAL,25600.0,41114.4,NaN
1040573,1040574,11382,981210,CREDIT,COLLECTION FROM ANOTHER BANK,46248.0,87362.4,NaN
1050361,1050362,11382,981225,DEBIT,CASH WITHDRAWAL,6300.0,81062.4,NaN
1053036,1053037,11382,981231,CREDIT,NaN,311.3,81373.6,INTEREST CREDITED


In [8]:
new_client[tr_by_acct["account_id"]]

KeyError: "None of [Int64Index([    1,     1,     1,     1,     1,     1,     1,     1,     1,\n                1,\n            ...\n            11382, 11382, 11382, 11382, 11382, 11382, 11382, 11382, 11382,\n            11382],\n           dtype='int64', length=1056320)] are in the [columns]"

In [9]:
new_client

,Unnamed: 0,client_id,birth_number,district_id,gender,age,age_levels
0,1,1,701213,18,FEMALE,29,ADULT
1,2,2,450204,1,MALE,54,MIDDLE AGED
2,3,3,401009,1,FEMALE,59,MIDDLE AGED
3,4,4,561201,5,MALE,43,MIDDLE AGED
4,5,5,600703,5,FEMALE,39,MIDDLE AGED
...,...,...,...,...,...,...,...
5364,5365,13955,451030,1,FEMALE,54,MIDDLE AGED
5365,5366,13956,430406,1,MALE,56,MIDDLE AGED
5366,5367,13968,680413,61,MALE,31,ADULT
5367,5368,13971,621019,67,FEMALE,37,MIDDLE AGED


In [10]:
len(tr_by_acct), len(new_disposition)

(1056320, 5369)

In [11]:
ud = new_disposition["account_id"].unique()
ut = tr_by_acct["account_id"].unique()

In [12]:
len(ud), len(ut)

(4500, 4500)

In [13]:
sum(ud-ut)

0

In [14]:
from datetime import datetime
czech_date_parser = lambda x: datetime.strptime(str(x), "%y%m%d")


In [15]:
df = pd.merge(
        pd.merge(
            tr_by_acct,
            new_disposition[new_disposition["type"]=="OWNER"][["account_id", "client_id"]],
            on="account_id",
        ),
        new_client[["client_id", "age"]],
        on = "client_id"
)
df

,Unnamed: 0,account_id,date,type,operation,amount,balance,k_symbol,client_id,age
0,149432,1,950324,CREDIT,CREDIT IN CASH,1000.0,1000.0,NaN,1,29
1,157404,1,950413,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,4679.0,NaN,1,29
2,158832,1,950423,CREDIT,CREDIT IN CASH,12600.0,17279.0,NaN,1,29
3,162681,1,950430,CREDIT,NaN,19.2,17298.2,INTEREST CREDITED,1,29
4,167083,1,950513,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,20977.2,NaN,1,29
...,...,...,...,...,...,...,...,...,...,...
1056315,1033141,11382,981202,DEBIT,CASH WITHDRAWAL,25600.0,41114.4,NaN,13998,46
1056316,1040574,11382,981210,CREDIT,COLLECTION FROM ANOTHER BANK,46248.0,87362.4,NaN,13998,46
1056317,1050362,11382,981225,DEBIT,CASH WITHDRAWAL,6300.0,81062.4,NaN,13998,46
1056318,1053037,11382,981231,CREDIT,NaN,311.3,81373.6,INTEREST CREDITED,13998,46


In [16]:
df["datetime"] = df["date"].apply(czech_date_parser)
df

,Unnamed: 0,account_id,date,type,operation,amount,balance,k_symbol,client_id,age,datetime
0,149432,1,950324,CREDIT,CREDIT IN CASH,1000.0,1000.0,NaN,1,29,1995-03-24
1,157404,1,950413,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,4679.0,NaN,1,29,1995-04-13
2,158832,1,950423,CREDIT,CREDIT IN CASH,12600.0,17279.0,NaN,1,29,1995-04-23
3,162681,1,950430,CREDIT,NaN,19.2,17298.2,INTEREST CREDITED,1,29,1995-04-30
4,167083,1,950513,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,20977.2,NaN,1,29,1995-05-13
...,...,...,...,...,...,...,...,...,...,...,...
1056315,1033141,11382,981202,DEBIT,CASH WITHDRAWAL,25600.0,41114.4,NaN,13998,46,1998-12-02
1056316,1040574,11382,981210,CREDIT,COLLECTION FROM ANOTHER BANK,46248.0,87362.4,NaN,13998,46,1998-12-10
1056317,1050362,11382,981225,DEBIT,CASH WITHDRAWAL,6300.0,81062.4,NaN,13998,46,1998-12-25
1056318,1053037,11382,981231,CREDIT,NaN,311.3,81373.6,INTEREST CREDITED,13998,46,1998-12-31


In [26]:
df.to_csv("data/tr_by_acct_w_age.csv", index=False)